In [1]:
# %pip install geojson
%pip install pymongo
%pip install certifi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
# data libaries
import numpy as np
import pandas as pd
import math

In [14]:
# data type libaries
from datetime import datetime
from geojson import Feature

# Sample Trajectories

In [10]:
# Define Trajectory 1
passenger = [
    [40.747776, -73.987974, "2024-04-09T11:00:00Z"],
    [40.748776, -73.986974, "2024-04-09T11:05:00Z"],
    [40.749776, -73.985974, "2024-04-09T11:10:00Z"],
    [40.750776, -73.984974, "2024-04-09T11:15:00Z"],
]

drivers = [
    [
        [40.747876, -73.988074, "2024-04-09T11:00:00Z"],
        [40.748876, -73.987074, "2024-04-09T11:05:00Z"],
        [40.749876, -73.986074, "2024-04-09T11:10:00Z"],
        [40.750876, -73.985074, "2024-04-09T11:15:00Z"],
    ],
    [
        [40.747976, -73.988174, "2024-04-09T11:00:00Z"],
        [40.748976, -73.987174, "2024-04-09T11:05:00Z"],
        [40.749976, -73.986174, "2024-04-09T11:10:00Z"],
        [40.750976, -73.985174, "2024-04-09T11:15:00Z"],
    ],
]

driver1 = [
    [40.747876, -73.988074, "2024-04-09T11:00:00Z"],
    [40.748876, -73.987074, "2024-04-09T11:05:00Z"],
    [40.749876, -73.986074, "2024-04-09T11:10:00Z"],
    [40.750876, -73.985074, "2024-04-09T11:15:00Z"],
]

driver2 = [
    [40.747976, -73.988174, "2024-04-09T11:00:00Z"],
    [40.748976, -73.987174, "2024-04-09T11:05:00Z"],
    [40.749976, -73.986174, "2024-04-09T11:10:00Z"],
    [40.750976, -73.985174, "2024-04-09T11:15:00Z"],
]

In [16]:
# data pulled from database
sample_request = {
  "_id": {
    "$oid": "661fc5c0bc83e7536732d789"
  },
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -63.988074,
          40.747876
        ]
      },
      "properties": {
        "time": "2024-04-09T11:00:00Z"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -73.987074,
          40.748876
        ]
      },
      "properties": {
        "time": "2024-04-09T11:05:00Z"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -73.986074,
          40.749876
        ]
      },
      "properties": {
        "time": "2024-04-09T11:10:00Z"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          -73.985074,
          40.750876
        ]
      },
      "properties": {
        "time": "2024-04-09T11:15:00Z"
      }
    }
  ]
}

### Data Conversion

In [6]:
def convert_to_unix(timestamp) -> float:
    datetime_obj = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
    unix_time = datetime_obj.timestamp()
    return unix_time

In [12]:
def convert_data(matrix):
    # Convert the timestamps to Unix time using pandas (you could also use datetime module in Python)
    timestamps = [convert_to_unix(date) for _ , _ , date in matrix]

    # Create a numpy array for latitudes, longitudes, and Unix timestamps
    latitudes = np.array([lat for lat, _, _ in matrix])
    longitudes = np.array([lng for _, lng, _ in matrix])
    timestamps = np.array(timestamps)

    return np.array(list(zip(latitudes, longitudes, timestamps)))


In [17]:
# convert geojson point features to numpy.ndarray
def geojson_to_ndarray(geojson_data):
    try:
        data = []
        for feature in geojson_data["features"]:
            latitude = feature["geometry"]["coordinates"][1]
            longitude = feature["geometry"]["coordinates"][0]
            time = feature["properties"]["time"]
            unix_time = convert_to_unix(time)
            data.append([latitude, longitude, unix_time])

        if not data:
            return None

        return np.array(data)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [20]:
## Example Conversion
convert_data(passenger)

array([[ 4.0747776e+01, -7.3987974e+01,  1.7126604e+09],
       [ 4.0748776e+01, -7.3986974e+01,  1.7126607e+09],
       [ 4.0749776e+01, -7.3985974e+01,  1.7126610e+09],
       [ 4.0750776e+01, -7.3984974e+01,  1.7126613e+09]])

In [21]:
geojson_to_ndarray(sample_request)

array([[ 4.0747876e+01, -6.3988074e+01,  1.7126604e+09],
       [ 4.0748876e+01, -7.3987074e+01,  1.7126607e+09],
       [ 4.0749876e+01, -7.3986074e+01,  1.7126610e+09],
       [ 4.0750876e+01, -7.3985074e+01,  1.7126613e+09]])

## Testing of Dynamic Time Warping Libraries

### TSLearn

In [143]:
from tslearn.metrics import dtw

# Numpy Matrix
ts1 = np.random.random((100,3))
ts2 = np.random.random((150,3))

# TSLean DTW test
distance = dtw(ts1, ts2)

6.0401025336665315

In [144]:
# trajectory data to np
passenger_data = convert_data(passenger)
driver_data_1 = convert_data(driver1)
driver_data_2 = convert_data(driver2)

In [145]:
dist1 = dtw(passenger_data, driver_data_1)
dist2 = dtw(passenger_data, driver_data_2)

In [147]:
print(f"Distance from passenger to driver 1: {dist1}")
print(f"Distance from passenger to driver 2: {dist2}")

Distance from passenger to driver 1: 0.0002828427124789841
Distance from passenger to driver 2: 0.0005656854249529439


### GPS point interpolation

In [15]:
# Calculate the distance between two points on the Earth's surface using Haversine formula.
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    
    """
    R = 6371  # Radius of the Earth in kilometers
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

In [29]:
# Compute intermediate points along the trajectory between two points.
def intermediate_points(lat1, lon1, lat2, lon2, n):
    intermediate_points = []
    total_distance = haversine_distance(lat1, lon1, lat2, lon2)
    segment_distance = total_distance / (n + 1)  # n+2 points including the start and end points
    for i in range(1, n + 1):
        f = segment_distance * i / total_distance
        lat = lat1 + f * (lat2 - lat1)
        lon = lon1 + f * (lon2 - lon1)
        intermediate_points.append((lat, lon))
    return intermediate_points

In [30]:
# passenger 1 trajectory: 55.463125, 10.089167 -> 55.535027, 10.096811
p1_start = [55.463125, 10.089167]
p1_end = [55.535027, 10.096811]

# passenger 2 trajectory: 55.502454, 10.076493 -> 55.496300, 10.109686
p2_start = [55.502454, 10.076493]
p2_end = [55.496300, 10.109686]

# cross point: 55.498474, 10.096576
cross_point = [55.498474, 10.096576]

In [31]:
p1_points = intermediate_points(p1_start[0], p1_start[1], p1_end[0], p1_end[1], 10)

In [18]:
p2_points = intermediate_points(p2_start[0], p2_start[1], p2_end[0], p2_end[1], 10)

In [35]:
p1_points

[(55.46966154545454, 10.089861909090908),
 (55.47619809090909, 10.090556818181819),
 (55.48273463636364, 10.091251727272727),
 (55.48927118181818, 10.091946636363636),
 (55.49580772727273, 10.092641545454546),
 (55.50234427272727, 10.093336454545454),
 (55.508880818181815, 10.094031363636365),
 (55.51541736363636, 10.094726272727273),
 (55.52195390909091, 10.095421181818182),
 (55.528490454545455, 10.096116090909092)]

In [34]:
p2_points

[(55.50189454545455, 10.079510545454545),
 (55.50133509090909, 10.08252809090909),
 (55.500775636363635, 10.085545636363635),
 (55.50021618181818, 10.08856318181818),
 (55.49965672727273, 10.091580727272726),
 (55.49909727272727, 10.094598272727273),
 (55.498537818181816, 10.097615818181819),
 (55.49797836363636, 10.100633363636364),
 (55.49741890909091, 10.10365090909091),
 (55.49685945454545, 10.106668454545455)]

In [38]:
# Find the closest pair of points between two sets of points.
def closest_pair(points1, points2):
    min_distance = float('inf')
    closest_point_pair = None
    for p1 in points1:
        for p2 in points2:
            d = haversine_distance(p1[0], p1[1], p2[0], p2[1])
            if d < min_distance:
                min_distance = d
                closest_point_pair = (p1, p2)
    return closest_point_pair, min_distance

In [37]:
closest_pair, min_distance = closest_pair(p1_points, p2_points)
print("Closest pair of points:", closest_pair)
print("Minimum distance:", min_distance)

Closest pair of points: ((55.50234427272727, 10.093336454545454), (55.49965672727273, 10.091580727272726))
Minimum distance: 0.3186426547212403
